# Lab02: Frequent itemset mining

- Student ID: 20120313
- Student name: Phan Tấn Kiệt

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [1]:
from collections import defaultdict

### Read data
First we have to read data from database

### Tree Projection

In [2]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**  
Apriori sử dụng phương pháp kết hợp và cắt bỏ các tập hợp ứng viên dựa trên giá trị support. Thuật toán này sử dụng cấu trúc candidate itemsets để lưu trữ các tập hợp ứng viên và tiếp tục sinh ra các frequent itemsets cho đến khi không còn tập hợp nào thỏa mãn điều kiện support nữa.  
  
Tree Projection là một phương pháp cải tiến của Apriori. Thuật toán này sử dụng cấu trúc cây để lưu trữ dữ liệu dạng transaction và xây dựng các frequent itemsets. Các frequent itemsets được sinh ra từ cây mà không cần phải thực hiện quá nhiều các phép kết hợp và cắt bỏ như trong Apriori.

In [3]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    sp =0
    t1= a.copy()
    t2 = b.copy()
    for itemset in data.items():
        if set(a['itemset']+b['itemset']).issubset(itemset[1]):
            sp+=1
    t2['itemset'] = sorted(list(set(t1['itemset']+t2['itemset'])))
    t2['pruned'] = False
    t2['support'] = sp
    ret = t1  
    difference = list(set(t2['itemset'])-set(t1['itemset']))
    ret[difference[0]] = t2
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?
        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}
        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}
            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.
            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.
            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''
        return tree, search_space

    def computeItemsetSupport(self, itemset):
        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        return itemset['support']
    

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        frequent_itemsets = {}
        non_frequent_itemsets = {}
        for itemset in search_space.items():
            # Kiểm tra độ support của frequent itemset
            support = self.computeItemsetSupport(itemset[1].copy())
            if support >= self.minSup:
                frequent_itemsets[itemset[0]] = itemset[1].copy()
            else:
                non_frequent_itemsets[itemset[0]] = itemset[1].copy()
        # Thêm các tập phổ biến vào danh sách L[k]
        temp =[]
        for itemset in frequent_itemsets.values():
            self.L[k].append(itemset['itemset'].copy())
        # Cắt các tập không phổ biến trong không gian tìm kiếm
        for itemset in non_frequent_itemsets:
            search_space[itemset]['pruned'] =True

    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue
            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).
                # TODO:
                # First create newset using join set
                newset = joinset(search_space[a],search_space[b])
                # Second add newset to search_space
                search_space[a]=newset
                sub_search_space[b] = search_space[a][b]
                sub_tree[b] ={}
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k+1, sub_tree, sub_search_space)
                

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [4]:
data, s= readData('chess.txt')

In [5]:
#
a=TP(data=data,s=s, minSup=3000)

print(a.miningResults())

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [[48, 52], [48, 58], [29, 56], [52, 56], [56, 58], [60, 66], [29, 66], [52, 66], [58, 66], [34, 40], [29, 34], [34, 52], [34, 58], [60, 62], [40, 62], [29, 62], [52, 62], [58, 62], [7, 60], [7, 40], [7, 29], [7, 52], [7, 58], [36, 60], [36, 40], [29, 36], [36, 52], [36, 58], [40, 60], [29, 60], [52, 60], [58, 60], [29, 40], [40, 52], [40, 58], [29, 52], [29, 58], [52, 58]], 3: [[48, 52, 58], [29, 52, 56], [29, 56, 58], [52, 56, 58], [29, 60, 66], [52, 60, 66], [58, 60, 66], [29, 52, 66], [29, 58, 66], [52, 58, 66], [29, 34, 40], [34, 40, 52], [34, 40, 58], [29, 34, 52], [29, 34, 58], [34, 52, 58], [29, 60, 62], [52, 60, 62], [58, 60, 62], [29, 40, 62], [40, 52, 62], [40, 58, 62], [29, 52, 62], [29, 58, 62], [52, 58, 62], [7, 40, 60], [7, 29, 60], [7, 52, 60], [7, 58, 60], [7, 29, 40], [7, 40, 52], [7, 40, 58], [7, 29, 52], [7, 29, 58], [7, 52, 58], [36, 40, 60], [29, 36, 60], [36, 52, 60], [36, 58, 60], [29

### Answer questions here:
**Why don't we compute support of items while reading data?**  
Ta không tính support khi đọc dữ liệu vì sẽ gây tốn chi phí, tính toán những phép tính không cần thiết và làm giảm hiệu suất của thuật toán.  
  
**why should we do sort**  
Để các phần tử trong self.s được sắp xếp theo thứ tự tăng dần của support, giúp xây dựng danh sách các frequent itemset theo thứ tự từ support thấp đến support cao. Việc sắp xếp này đảm bảo frequent itemsets với support thấp được kiểm tra trước, giảm số lần duyệt database và tối ưu hóa thuật toán khai thác dữ liệu.  
  
**study about python set and its advantages ?**  
Khi lưu trữ itemsets trong các thuật toán dạng cây như Apriori và FP, thường sử dụng Python set vì nó có một số lợi ích sau:  
1. Không trùng lặp: Set trong Python không cho phép các phần tử trùng lặp, sử dụng set giúp đơn giản hóa quá trình xử lý dữ liệu và đảm bảo tính chính xác trong việc tính toán support, confidence cho các frequent itemsets.   
  
2. Nhanh và hiệu quả: Set trong Python được cài đặt dưới dạng bảng băm do đó việc truy vấn, thêm, xóa các phần tử trong set được thực hiện nhanh chóng giúp cải thiện hiệu suất của thuật toán khai thác dữ liệu, đặc biệt là trong việc kiểm tra sự tồn tại của một itemset trong tập dữ liệu lớn.  
  
3. Linh hoạt và dễ sử dụng: Set trong Python cung cấp nhiều hàm hữu ích cho việc xử lý dữ liệu, chẳng hạn như phép hợp (union), phép giao (intersection), phép hiệu (difference), phép kiểm tra tập con (issubset), phép kiểm tra tập phụ thuộc (issuperset),... Điều này giúp đơn giản hóa việc thao tác và xử lý các itemset.  
  
**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**  
1. Để bảo vệ tính toàn vẹn của dữ liệu, trong trường hợp sử dụng search_space và tree trong các bước tiếp theo hoặc để phân tích thêm.  
2. Việc giữ lại giúp chúng ta có thể truy xuất hay quan sát tốt hơn, nó có thể hữu ích khi tìm hiểu cách hoạt động của thuật toán và để xác định sự cố hay lỗi tiềm ẩn.  
3. Có thể sử dụng lại.  
4. Tiết kiệm chi phí.  

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**  
Apriori sử dụng phương pháp kết hợp và cắt bỏ các tập hợp ứng viên dựa trên giá trị support. Thuật toán này sử dụng cấu trúc candidate itemsets để lưu trữ các tập hợp ứng viên và tiếp tục sinh ra các frequent itemsets cho đến khi không còn tập hợp nào thỏa mãn điều kiện support nữa.  
  
Tree Projection là một phương pháp cải tiến của Apriori. Thuật toán này sử dụng cấu trúc cây để lưu trữ dữ liệu dạng transaction và xây dựng các frequent itemsets. Các frequent itemsets được sinh ra từ cây mà không cần phải thực hiện quá nhiều các phép kết hợp và cắt bỏ như trong Apriori.


# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

In [6]:
def readChurn(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=0;
        next(f)
        for line in f:
            line = line[:-2]
            itemset=set(line.split(",")) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    return data, s

In [7]:
dataChurn, sChurn= readChurn('churn.txt')

In [8]:
a=TP(data=dataChurn,s=sChurn, minSup=100)

print(a.miningResults())

{1: [['75'], ['127'], ['126'], ['WV'], ['73'], ['124'], ['9'], ['76'], ['8'], ['81'], ['125'], ['79'], ['74'], ['77'], ['122'], ['119'], ['78'], ['82'], ['84'], ['118'], ['80'], ['121'], ['123'], ['120'], ['85'], ['83'], ['116'], ['86'], ['114'], ['111'], ['89'], ['115'], ['117'], ['113'], ['112'], ['90'], ['109'], ['87'], ['110'], ['103'], ['107'], ['93'], ['99'], ['88'], ['96'], ['92'], ['97'], ['101'], ['91'], ['106'], ['100'], ['98'], ['95'], ['104'], ['7'], ['108'], ['94'], ['102'], ['105'], ['6'], ['True'], ['5'], ['4'], ['408'], ['510'], ['3'], ['2'], ['yes'], ['1'], ['415'], ['0'], ['False'], ['no']], 2: []}


# 4 References

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
